In [ ]:
pip install keras-tuner --upgrade

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import  Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout, Resizing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner
import itertools
import os
import random
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
if os.path.isdir('output') is False:
    !pip install split-folders
    import splitfolders
    splitfolders.ratio('/kaggle/input/trainingdatafinal/training_data_final',output='output',seed=1337,ratio=(0.8,0.2))

In [ ]:
train_path="./output/train"
valid_path="./output/val"

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

train_data_gen = ImageDataGenerator(rescale=1./255.,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    brightness_range=[.2, 1.2],
                                    fill_mode="reflect",
                                   )

valid_data_gen = ImageDataGenerator(rescale=1./255.)

train_gen = train_data_gen.flow_from_directory(directory=train_path,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed
                                              )

valid_gen = valid_data_gen.flow_from_directory(directory=valid_path,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed
                                              )


In [ ]:
speciesCount=[]
for i in range(8):
  file_count = sum(len(files) for _, _, files in os.walk('../input/trainingdatafinal/training_data_final/Species'+str(i+1)))
  speciesCount.append(file_count)

sumc=0
for num in speciesCount:
  sumc=sumc+num
avg=sumc/8

class_weights={}
for i in range(len(speciesCount)):
  class_weights[i]=avg/speciesCount[i]

class_weights[0]=class_weights[0]
class_weights[5]=class_weights[5]
class_weights

In [ ]:
base_model = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3),
    )
base_model.trainable = False

In [ ]:
def model_skeleton(units1, units2, dropout1, dropout2, lr):
    model = keras.Sequential()
    inputs = Input(shape=(96, 96, 3))
    model.add(inputs)
    model.add(base_model)
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(units1,activation='relu'))
    model.add(keras.layers.Dropout(dropout1, seed=seed))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(units2,activation='relu'))
    model.add(keras.layers.Dropout(dropout2, seed=seed))
    model.add (keras.layers.Dense(8, activation='softmax'))
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
def build_model(hp):
    units1 = hp.Int("units1", min_value=512, max_value=1024, step=32)
    units2 = hp.Int("units2", min_value=32, max_value=512, step=32)
    dropout1 = hp.Float("dropout1", min_value=0.5, max_value=0.7, step=0.05)
    dropout2 = hp.Float("dropout2", min_value=0.4, max_value=0.6, step=0.05)
    lr = hp.Float("lr", min_value=1e-6, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = model_skeleton(
        units1=units1,units2=units2, dropout1=dropout1, dropout2=dropout2, lr=lr
    )
    return model

In [ ]:
build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=30,
    overwrite=True,
    directory="HPT",
    project_name="DenNet201HPT",
    )

In [ ]:
tuner.search(train_gen, epochs=5, validation_data=valid_gen)

In [ ]:
model_list = tuner.get_best_models(num_models = 2)
best_model = model_list[0]

best_model.build()
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(2)
# Build the model with the best hp.
model = build_model(best_hps[0])

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=5, verbose=1)
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
check_point = keras.callbacks.ModelCheckpoint(filepath="models/CNN.h5",monitor="val_accuracy",mode="max",save_best_only=True)
history=model.fit(x=train_gen, validation_data=valid_gen, epochs=100,callbacks=[reduce_lr, es_callback, check_point],class_weight=class_weights, verbose=2).history

In [ ]:
model_CNNTF = tf.keras.models.load_model("models/CNN.h5")

In [ ]:
def model_skeleton_finetuning (lr):
    # Build the model with the best hp.
    model = model_CNNTF
    model.get_layer('densenet201').trainable = True
    model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def build_modelFT(hp):
    lr = hp.Float("lr", min_value=1e-6, max_value=1e-3, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = model_skeleton_finetuning(lr = lr)
    return model

In [ ]:
build_modelFT(keras_tuner.HyperParameters())

In [ ]:
tunerFT = keras_tuner.BayesianOptimization(
    build_modelFT,
    objective='val_loss',
    max_trials=30,
    overwrite=True,
    directory="HPT",
    project_name="DenNet201FTHPT",
    )

In [ ]:
tunerFT.search(train_gen, epochs=5, validation_data=valid_gen)

In [ ]:
tunerFT.results_summary()

In [ ]:
# Get the top 2 hyperparameters.
best_hpsFT = tunerFT.get_best_hyperparameters(5)
# Build the model with the best hp.
model_CNNFT = build_modelFT(best_hpsFT[0])

In [ ]:
check_point2 = keras.callbacks.ModelCheckpoint(filepath="models/CNNFT.h5",monitor="val_accuracy",mode="max",save_best_only=True)
historyFT=model_CNNFT.fit(x=train_gen, validation_data=valid_gen, epochs=100,callbacks=[reduce_lr, es_callback, check_point2],class_weight=class_weights, verbose=2).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Standard', alpha=.8, color='#ff7f0e')
plt.plot(historyFT['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(historyFT['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(historyFT['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(historyFT['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
model_saved=tf.keras.models.load_model("models/CNNFT.h5")
model_saved.save("model")